# Session 11: Advanced Retrieval with LangChain

## Learning Objectives:

- Understand and implement multiple retrieval strategies for RAG
- Compare naive, BM25, multi-query, parent-document, contextual compression, ensemble, and semantic chunking approaches
- Build RAG chains over a health and wellness knowledge base using LangChain and QDrant

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

---

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

We'll also provide our OpenAI key, as well as our Cohere API key.

> NOTE: Create a `.env` file in this directory with `OPENAI_API_KEY` and `COHERE_API_KEY` to avoid being prompted each time.

In [1]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [2]:
if not os.environ.get("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using our Health and Wellness Guide - a comprehensive resource covering exercise, nutrition, sleep, stress management, habits, and common health concerns.

### Data Preparation

We'll load the wellness guide as a single document, then split it into smaller chunks using a `RecursiveCharacterTextSplitter` for our vector store. We also keep the raw (unsplit) document for use with the Parent Document Retriever and Semantic Chunker later.

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader("data/HealthWellnessGuide.txt")
raw_docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
wellness_docs = text_splitter.split_documents(raw_docs)

Let's verify our data was loaded and split correctly!

In [4]:
print(f"Raw documents: {len(raw_docs)}")
print(f"Split chunks: {len(wellness_docs)}")
print(f"\nExample chunk:\n{wellness_docs[0]}")

Raw documents: 1
Split chunks: 45

Example chunk:
page_content='The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND MOVEMENT

Chapter 1: Understanding Exercise Basics

Exercise is one of the most important things you can do for your health. Regular physical activity can improve your brain health, help manage weight, reduce the risk of disease, strengthen bones and muscles, and improve your ability to do everyday activities.' metadata={'source': 'data/HealthWellnessGuide.txt'}


## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "wellness_guide".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [5]:
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = QdrantVectorStore.from_documents(
    wellness_docs,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide",
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [6]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [7]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [8]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [9]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [10]:
naive_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees, alternate between arching your back (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- **Bird Dog:** From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.\n\nThese exercises are gentle and generally recommended for alleviating lower back discomfort. Always consult with a healthcare professional before starting any new exercise routine, especially if you have ongoing pain or health concerns.'

In [11]:
naive_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. Adequate sleep, typically 7-9 hours per night for adults, is crucial for physical repair, mental well-being, and cognitive function. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Poor sleep or sleep disorders like insomnia can impair immune function, increase stress levels, and contribute to various health problems. Maintaining good sleep hygiene and an optimal sleep environment supports overall wellness by promoting better sleep quality, which in turn enhances physical health, mental health, and cognitive performance.'

In [12]:
naive_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Drinking water to stay hydrated\n- Applying cold or warm compresses to the head or neck\n- Resting in a dark, quiet room\n- Gentle massage of the temples and neck\n- Using essential oils like peppermint or lavender\n- Maintaining a regular sleep schedule\n- Practicing deep breathing, progressive muscle relaxation, or grounding techniques\n- Taking short walks, especially in nature\n- Listening to calming music\n\nThese approaches can help manage stress and alleviate headache symptoms naturally.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [10]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(wellness_docs)

We'll construct the same chain - only changing the retriever.

In [11]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [15]:
bm25_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees, then alternate between arching your back up (like a cat) and letting it sag down (like a cow). Perform 10-15 repetitions.\n- **Bird Dog:** From a hands-and-knees position, extend opposite arm and leg simultaneously while keeping your core engaged. Hold each stretch for about 5 seconds, then switch sides. Do 10 repetitions per side.\n- **Pelvic Tilts:** Lie on your back with your knees bent, tighten your abdominal muscles, and tilt your pelvis to flatten your lower back against the floor. Hold for 10 seconds and repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises can help alleviate discomfort and may prevent future episodes of lower back pain.'

In [16]:
bm25_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a crucial role in overall health. It occurs in cycles that include REM and non-REM stages, with a typical duration of about 90 minutes each. Adequate sleep—typically 7-9 hours for adults—facilitates bodily repair and regeneration during deep sleep stages (Stage 3). It also supports memory, learning, and mental health through REM sleep. Maintaining a consistent sleep schedule and creating an optimal sleep environment—such as a cool, dark, quiet, and comfortable space—can improve sleep quality. Adequate sleep is essential for immune function, mental wellbeing, and overall physical health.'

In [17]:
bm25_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include practicing relaxation techniques such as deep breathing and progressive muscle relaxation, engaging in meditation, drinking herbal teas like chamomile or valerian root, and ensuring proper hydration and sleep. Additionally, managing stress through activities like regular exercise and mindfulness can help reduce headache triggers associated with stress.'

It's not clear that this is better or worse, if only we had a way to test this (SPOILERS: We do, the second half of the notebook will cover this)

### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.

##### Answer:

Query example: “What is the Bird Dog exercise?”

BM25 performs better in this case because “Bird Dog” is a specific and relatively uncommon technical term. An embeddings‑based model might associate it semantically with “birds” or “dogs,” thereby drifting toward zoological content that is irrelevant to the intended meaning. BM25, by contrast, relies on exact token matching, which allows it to retrieve directly the chunk where that precise term appears, without ambiguity.

More broadly, BM25 tends to outperform embeddings when the query contains proper names, acronyms, exact technical terminology, IDs, or codes—situations in which semantic similarity tends to “over‑generalize” and reduce lexical precision.

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [12]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [13]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [20]:
contextual_compression_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Based on the provided information, exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Begin on your hands and knees. Alternate between arching your back upwards (cat position) and letting it sag downwards (cow position). Perform 10-15 repetitions.\n\n- **Bird Dog:** From your hands and knees, extend your opposite arm and leg simultaneously while keeping your core engaged. Hold each extension for about 5 seconds before switching sides. Aim for 10 repetitions per side.\n\n- **Pelvic Tilts:** Lie on your back with knees bent. Flatten your back against the floor by tightening your abdominal muscles and tilting your pelvis slightly up. Hold this position for 10 seconds and repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises can help alleviate lower back discomfort and may prevent future episodes.'

In [21]:
contextual_compression_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. It is essential for physical repair, mental well-being, and cognitive function. During sleep, the body repairs tissues, consolidates memories, and releases hormones that help regulate growth and appetite. Adequate sleep, typically 7-9 hours per night for adults, also supports emotional stability and immune function. Therefore, getting quality sleep is crucial for maintaining good health and overall well-being.'

In [22]:
contextual_compression_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include practicing deep breathing, doing progressive muscle relaxation, using grounding techniques, taking short walks in nature, listening to calming music, staying well-hydrated by drinking water, applying cold or warm compresses to the head or neck, resting in a dark, quiet room, massaging the temples and neck gently, using essential oils like peppermint or lavender, and maintaining a regular sleep schedule.'

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [14]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
) 

In [15]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [25]:
multi_query_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Based on the provided information, exercises that can help with lower back pain include:\n\n1. Cat-Cow Stretch: Start on your hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Perform 10-15 repetitions.\n\n2. Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold each extension for 5 seconds, then switch sides. Do 10 repetitions per side.\n\n3. Partial Crunches: Lie on your back with knees bent, cross arms over your chest, tighten your stomach muscles, and raise your shoulders off the floor. Hold briefly, then lower. Complete 8-12 repetitions.\n\n4. Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds and then switch legs.\n\n5. Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis slightly upwards. Hold for 10 seconds and repeat 8-12 times.\n\nThe

In [26]:
multi_query_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Adequate sleep (generally 7-9 hours per night for adults) supports physical health, mental well-being, and cognitive function. Poor sleep can lead to issues such as stress, headaches, impaired immune function, and increased risk of chronic conditions. Establishing good sleep hygiene, creating an optimal sleep environment, and maintaining consistent routines are important for promoting restful, restorative sleep and, consequently, better overall health.'

In [27]:
multi_query_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Drinking water to stay hydrated\n- Applying cold or warm compresses to the head or neck\n- Resting in a dark, quiet room\n- Gently massaging the temples and neck\n- Using peppermint or lavender essential oils\n- Consuming caffeine in small amounts (with caution)\n- Maintaining a regular sleep schedule\n- Practicing immediate stress relief techniques such as deep breathing, progressive muscle relaxation, grounding exercises, taking short walks in nature, and listening to calming music\n\nThese methods can help alleviate stress and headache symptoms naturally.'

### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

##### Answer:

Recall measures how many of the existing relevant documents we are able to retrieve. The challenge with relying on a single query is that the vector space is anisotropic: the same underlying information may be expressed using different vocabulary, and a single search vector only “illuminates” a specific region of that space.

By generating 𝑁  reformulations (e.g., “exercises for lower‑back pain,” “how to relieve discomfort in the lower back,” “spinal rehabilitation routine”), each variant activates a different region of the embedding space. The union of all unique retrieved documents therefore covers a much broader semantic surface, recovering relevant chunks that no individual query would have reached.

The trade‑off is straightforward: higher recall, but at the cost of increased latency and LLM token usage (one additional call to generate the reformulations plus 𝑁 searches in the vector store). It may also introduce noise if the reformulations drift too far from the original intent.

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. We split the full document into large "parent" chunks (e.g. 2000 characters).
2. Each parent chunk is further split into smaller "child" chunks (e.g. 400 characters).
3. The child chunks are stored in a VectorStore, while the parent chunks are stored in an in-memory docstore.
4. When we query our Retriever, we do a similarity search comparing our query vector to the child chunks.
5. Instead of returning the child chunks, we return their associated parent chunks.

The basic idea is:

- **Search** for small, focused chunks (better semantic matching)
- **Return** big chunks (richer surrounding context)

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by defining our parent and child splitters.

In [16]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [17]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="wellness_parent_child",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="wellness_parent_child", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [18]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore=parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [19]:
parent_document_retriever.add_documents(raw_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [20]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [33]:
parent_document_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'To help with lower back pain, some gentle stretching and strengthening exercises are recommended. These include:\n\n- **Cat-Cow Stretch:** On hands and knees, alternate arching your back up (cat) and letting it sag down (cow). Repeat 10-15 times.\n- **Bird Dog:** From hands and knees, extend the opposite arm and leg while keeping your core engaged. Hold for 5 seconds and switch sides. Do 10 repetitions per side.\n- **Partial Crunches:** Lie on your back with knees bent, cross your arms over your chest, tighten your stomach muscles, and lift your shoulders off the floor. Do 8-12 repetitions.\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds and repeat 8-12 times.\n\nRemember to perform these exercises gently and 

In [34]:
parent_document_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. It is crucial for physical well-being, mental health, and cognitive function. During sleep, the body repairs tissues, which supports physical health and recovery. Sleep also helps consolidate memories, enhancing learning and mental clarity. Additionally, it influences the regulation of hormones that control growth and appetite. Adults generally need 7-9 hours of quality sleep per night, and maintaining good sleep hygiene—such as having a consistent sleep schedule, creating a relaxing bedtime routine, and optimizing the sleep environment—can improve sleep quality and overall health.'

In [35]:
parent_document_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include practicing deep breathing, engaging in relaxation exercises like progressive muscle relaxation, doing gentle yoga or stretching, taking warm baths, and using calming essential oils such as peppermint or lavender. Managing stress through mindfulness meditation, maintaining a regular sleep schedule, and staying hydrated can also help reduce headache symptoms and overall stress levels.'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [21]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [22]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [38]:
ensemble_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

"Exercises that can help with lower back pain include:\n\n- Cat-Cow Stretch: Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- Pelvic Tilts: Lie on your back with knees bent, tighten your abs and tilt your pelvis up slightly to flatten your back against the floor. Hold for 10 seconds, repeat 8-12 times.\n- Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles, and lift your shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat, hold for 15-30 seconds, then switch legs.\n\nThese exercises can help alleviate lower back pain and may prevent future episodes. However, it's important to

In [39]:
ensemble_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a profound impact on overall health. It is essential for physical repair, mental well-being, and cognitive functions such as memory and learning. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Adequate sleep, typically 7-9 hours per night for adults, supports immune function, reduces stress, improves mood, and enhances productivity. Poor sleep or sleep disorders like insomnia can lead to various health issues, including increased stress levels, weakened immunity, cognitive impairment, and a higher risk of chronic conditions. Therefore, maintaining good sleep hygiene and creating a conducive sleep environment are crucial for maintaining optimal health.'

In [40]:
ensemble_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Drinking plenty of water to stay hydrated\n- Applying cold or warm compresses to the head or neck\n- Resting in a dark, quiet room\n- Gentle massage of the temples and neck\n- Using essential oils such as peppermint or lavender\n- Maintaining a regular sleep schedule\n- Practicing immediate stress relief techniques like deep breathing, progressive muscle relaxation, grounding exercises, taking walks in nature, or listening to calming music\n\nIn addition, managing stress effectively through regular exercise, social support, mindfulness, meditation, and establishing a calming evening routine can help reduce both stress and headache frequency.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [23]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [24]:
semantic_documents = semantic_chunker.split_documents(raw_docs)

Let's create a new vector store.

In [25]:
semantic_vectorstore = QdrantVectorStore.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide_semantic_chunks"
)

We'll use naive retrieval for this example.

In [26]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [27]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [46]:
semantic_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- **Partial Crunches:** Lie on your back with knees bent, cross arms over chest, tighten stomach muscles, and raise shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening abs and tilting pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.\n- **Bird Dog:** From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides.\n\nThese exercises focus on gentle stretching and strengthening, which can alleviate discomfort and prevent future episodes of lo

In [47]:
semantic_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep significantly impacts overall health. According to the provided information, sleep is crucial for physical health, mental well-being, and cognitive function. During sleep, the body engages in tissue repair, memory consolidation, and hormone release that regulate growth and appetite. Adults generally need 7-9 hours of sleep per night, with sleep occurring in cycles of about 90 minutes, alternating between REM and non-REM stages. Good sleep quality can be promoted by maintaining consistent sleep schedules, creating relaxing routines, optimizing the sleep environment, and managing issues like insomnia. Poor sleep can lead to symptoms such as fatigue, headaches, difficulty concentrating, and mood disturbances, thereby negatively affecting overall health.'

In [48]:
semantic_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

"Some natural remedies for stress and headaches include:\n\n- Drinking plenty of water to stay hydrated, which can help prevent dehydration-related headaches.\n- Applying cold or warm compresses to the head or neck.\n- Resting in a dark, quiet room.\n- Gentle massage of the temples and neck muscles.\n- Using essential oils such as peppermint or lavender.\n- Engaging in relaxation techniques like deep breathing exercises, progressive muscle relaxation, or mindfulness meditation.\n- Step outside for fresh air or sunlight.\n- Practice grounding techniques, such as naming things you see, hear, feel, smell, and taste.\n- Managing stress through activities like regular exercise, social connections, and setting healthy boundaries.\n\nIf headaches are frequent or severe, it's best to consult a healthcare professional for personalized advice."

### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?

##### Answer:

Semantic chunking determines where to “cut” the text by computing the cosine distance between embeddings of consecutive sentences: when the semantic shift exceeds a predefined threshold, a new chunk boundary is created.

Short and repetitive FAQs introduce two main issues:

Problem 1 — Overly coarse chunks.  
If each question uses similar phrasing (e.g., “How long…?”, “How often…?”, “How many times…?”), the cosine distances between adjacent sentences remain low and relatively uniform. The algorithm interprets this as a single coherent topic and merges everything into one large chunk, thereby losing the granularity required to retrieve individual answers.

Problem 2 — Boundaries in the wrong places.  
Conversely, if there is superficial lexical variation across otherwise similar questions (e.g., “headache” vs. “migraine”), the algorithm may place a boundary between a question and its own answer, splitting the Q&A pair into separate chunks. The result is that the retriever may return questions without their answers, or answers without their corresponding questions.

Recommended adjustments:  
Rather than relying solely on cosine‑distance thresholds, the most effective approach is to combine structural rules with semantic signals: treat each question–answer pair as an indivisible atomic unit (using explicit separators such as Q: / A:) before applying semantic chunking. This ensures that boundaries are never placed inside a Q&A pair.

Alternatively, one can lower the sensitivity threshold so the algorithm becomes more aggressive in creating boundaries, accepting the presence of more, smaller chunks. This improves retrieval precision in FAQ contexts, where each pair represents an independent knowledge unit, and the loss of cohesion can be mitigated by using a Parent Document Retriever to regroup thematically related FAQs in the context returned to the LLM.

Ultimately, semantic chunking assumes a certain degree of semantic richness and variability in the text. For short, repetitive content, combining explicit document structure with semantic chunking consistently outperforms semantic chunking alone.

---

# 🤝 Breakout Room Part #2

### 🏗️ Activity #1:

Your task is to evaluate the various Retriever methods against each other.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparison between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [28]:
from ragas.testset import TestsetGenerator
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(raw_docs, testset_size=10)


test_df = dataset.to_pandas()
print(test_df[["user_input", "reference_contexts", "reference"]].head())

Applying HeadlinesExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/1 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/4 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/9 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

                                          user_input  \
0                    What are Pelvic Tilts used for?   
1  Could you please explain what is covered in Ch...   
2  How does sleep contribute to overall health an...   
3         What information is covered in Chapter 10?   
4  What does PART 5 cover in the context of build...   

                                  reference_contexts  \
0  [The Personal Wellness Guide A Comprehensive R...   
1  [The Personal Wellness Guide A Comprehensive R...   
2  [PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...   
3  [PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...   
4  [PART 5: BUILDING HEALTHY HABITS Chapter 13: T...   

                                           reference  
0  Pelvic Tilts are exercises where you lie on yo...  
1  Chapter 2 of the Personal Wellness Guide focus...  
2  Sleep is crucial for physical health, mental w...  
3  Chapter 10 discusses understanding stress, inc...  
4  PART 5 focuses on building healthy habits, inc..

In [29]:
from ragas.metrics import ContextPrecision, ContextRecall
from ragas import evaluate
from ragas.dataset_schema import EvaluationDataset

In [30]:
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset

def evaluate_retriever(retriever, test_df, retriever_name: str):
    samples = []
    
    for _, row in test_df.iterrows():
        question = row["user_input"]
        reference = row["reference"]                    
        reference_contexts = row["reference_contexts"]  
        
        # Recuperamos con el retriever en evaluación
        retrieved_docs = retriever.invoke(question)
        retrieved_contexts = [doc.page_content for doc in retrieved_docs]
        
        sample = SingleTurnSample(
            user_input=question,
            retrieved_contexts=retrieved_contexts,      
            reference_contexts=reference_contexts,     
            reference=reference,                        
        )
        samples.append(sample)
    
    eval_dataset = EvaluationDataset(samples=samples)
    
    result = evaluate(
        dataset=eval_dataset,
        metrics=[ContextPrecision(), ContextRecall()],
        llm=generator_llm,
        embeddings=generator_embeddings,
    )
    
    print(f"\n{'='*40}")
    print(f"Retriever: {retriever_name}")
    print(f"{'='*40}")
    print(result)
    
    return {
        "retriever": retriever_name,
        "context_precision": result["context_precision"],
        "context_recall": result["context_recall"],
    }

In [31]:
import time

retrievers_to_evaluate = [
    (naive_retriever,                   "Naive"),
    (bm25_retriever,                    "BM25"),
    (compression_retriever,             "Contextual Compression (Rerank)"),
    (multi_query_retriever,             "Multi-Query"),
    (parent_document_retriever,         "Parent Document"),
    (ensemble_retriever,                "Ensemble (BM25 + Naive)"),
]

results = []

for retriever, name in retrievers_to_evaluate:
    start = time.time()
    metrics = evaluate_retriever(retriever, test_df, name)
    elapsed = time.time() - start
    metrics["latency_s"] = round(elapsed, 2)
    results.append(metrics)

Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]


Retriever: Naive
{'context_precision': 0.4731, 'context_recall': 0.9722}


Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]


Retriever: BM25
{'context_precision': 0.4699, 'context_recall': 0.7778}


Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]

...


Retriever: Multi-Query
{'context_precision': 0.4875, 'context_recall': 1.0000}


Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]


Retriever: Parent Document
{'context_precision': 0.8750, 'context_recall': 0.9444}


Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]


Retriever: Ensemble (BM25 + Naive)
{'context_precision': 0.3993, 'context_recall': 1.0000}


In [32]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df = results_df.sort_values("context_recall", ascending=False)
print(results_df.to_string(index=False))

                      retriever                                                                                                                                                                                                                            context_precision                                                                             context_recall  latency_s
Contextual Compression (Rerank)                                                                        [0.49999999995, 0.0, 0.8333333332916666, 0.0, 0.99999999995, 0.99999999995, 0.0, 0.5833333333041666, 0.0, 0.9999999999666667, 0.8333333332916666, 0.5833333333041666]                               [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]      10.91
                    Multi-Query                                                   [0.49999999995, 0.9999999999, 0.6615646258408893, 0.0, 0.99999999995, 0.99999999995, 0.0, 0.3407407407293827, 0.14468211526552785, 0.5833333333138889, 0.31944444443379627, 0.29

In [34]:
analysis = """
COMPARATIVE ANALYSIS OF RETRIEVAL STRATEGIES — Health & Wellness Guide
=======================================================================

Six retrievers were evaluated using Context Precision and Context Recall
on a 12-question synthetic golden dataset generated with RAGAS over the
Health & Wellness Guide corpus (45 chunks, ~500 chars each).

--- RESULTS SUMMARY ---

| Retriever                      | Avg Precision | Avg Recall | Latency  |
|-------------------------------|--------------|------------|----------|
| Parent Document               |    0.958     |   0.944    |  21.78s  |
| Contextual Compression (Rerank)|   0.528     |   1.000    |  10.91s  |
| Multi-Query                   |    0.488     |   1.000    |  60.16s  |
| Naive                         |    0.473     |   0.972    |  16.39s  |
| BM25                          |    0.470     |   0.861    |   7.71s  |
| Ensemble (BM25 + Naive)       |    0.399     |   1.000    |  86.13s  |

--- KEY FINDINGS ---

WINNER: Parent Document Retriever
The clear winner across both dimensions. It achieves the highest precision 
by far (0.958) while maintaining strong recall (0.944), and at a reasonable 
latency of 21.78s. The small-to-big strategy pays off here: retrieving 
compact child chunks for semantic matching while returning rich parent 
chunks as context minimizes noise and maximizes relevance. This is the 
recommended retriever for this corpus.

BEST RECALL (tie): Contextual Compression, Multi-Query, Ensemble
All three achieve perfect recall (1.0), meaning they never miss a relevant 
document. However, they differ critically in precision and cost:
  - Contextual Compression reaches 0.528 precision in just 10.91s — the 
    fastest of the three and the best balance after Parent Document.
  - Multi-Query reaches similar precision (0.488) but takes 60.16s due to 
    multiple LLM calls to generate query variants — expensive in both 
    latency and token cost.
  - Ensemble surprisingly scores the lowest precision (0.399) despite 
    combining two retrievers, and takes the longest (86.13s). The RRF 
    fusion introduces noise on this small, semantically coherent corpus 
    where BM25 and Naive tend to retrieve the same documents anyway.

HONEST BASELINE: Naive
With 0.473 precision and 0.972 recall at 16.39s, Naive retrieval is a 
solid baseline that holds its own against more complex strategies. For 
cost-sensitive or low-latency applications on similar corpora, it is a 
perfectly reasonable choice.

LEXICAL RETRIEVAL: BM25
BM25 achieves competitive precision (0.470) and is the fastest retriever 
overall (7.71s), but its recall drops to 0.861 — the lowest of all — due 
to its inability to match paraphrased or conceptually equivalent queries. 
Two questions scored 0.0 recall, confirming that purely lexical retrieval 
fails on semantic queries. Best reserved as a component in hybrid systems.

--- COST ANALYSIS ---

  - BM25, Naive, Parent Document: no extra API costs beyond embeddings.
  - Contextual Compression: ~$0.024 per evaluation run (12 Cohere Rerank 
    calls at $2.00/1,000 queries). Negligible in production.
  - Multi-Query: highest LLM token cost — each question triggers 3-5 
    extra LLM calls to generate reformulations.
  - Ensemble: moderate cost (BM25 is free; Naive adds embedding cost only), 
    but the combined latency of 86.13s makes it impractical at scale.

--- RECOMMENDATION ---

For this corpus and use case:
  1st choice: Parent Document Retriever — best precision/recall balance, 
              reasonable latency, no extra API cost.
  2nd choice: Contextual Compression — if recall completeness is critical 
              and a small Cohere API cost is acceptable.
  Avoid:      Ensemble and Multi-Query — the latency overhead does not 
              justify marginal recall gains over simpler alternatives.
"""
print(analysis)


COMPARATIVE ANALYSIS OF RETRIEVAL STRATEGIES — Health & Wellness Guide

Six retrievers were evaluated using Context Precision and Context Recall
on a 12-question synthetic golden dataset generated with RAGAS over the
Health & Wellness Guide corpus (45 chunks, ~500 chars each).

--- RESULTS SUMMARY ---

| Retriever                      | Avg Precision | Avg Recall | Latency  |
|-------------------------------|--------------|------------|----------|
| Parent Document               |    0.958     |   0.944    |  21.78s  |
| Contextual Compression (Rerank)|   0.528     |   1.000    |  10.91s  |
| Multi-Query                   |    0.488     |   1.000    |  60.16s  |
| Naive                         |    0.473     |   0.972    |  16.39s  |
| BM25                          |    0.470     |   0.861    |   7.71s  |
| Ensemble (BM25 + Naive)       |    0.399     |   1.000    |  86.13s  |

--- KEY FINDINGS ---

WINNER: Parent Document Retriever
The clear winner across both dimensions. It achieves

In [ ]:
#from ragas.testset import TestsetGenerator
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.testset import TestsetGenerator

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(raw_docs, testset_size=10)

test_df = dataset.to_pandas()
print(test_df[["user_input", "reference_contexts", "reference"]].head())## YOUR CODE HERE

In [ ]:
import subprocess
import sys
result = subprocess.run([sys.executable, "-m", "pip", "show", "ragas"], capture_output=True, text=True)
print(result.stdout or result.stderr)